In [10]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [11]:
import numpy as np
import os
import os.path
import pytest
import os.path as op
import sys
import matplotlib.pyplot as plt
import scipy
import glob

import imtools.trainer3d
import imtools.datasets
import imtools.ml
import io3d
import sed3

# Training

In [12]:
# Define feature function
def externfv(data3d, voxelsize_mm):        # scale
    f0 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=3).reshape(-1, 1)
    f1 = scipy.ndimage.filters.gaussian_filter(data3d, sigma=1).reshape(-1, 1) - f0
    fv = np.concatenate([
        f0, f1
    ], 1)
    return fv

In [13]:
sliver_reference_dir = op.expanduser("~/data/medical/orig/sliver07/training/")

ol = imtools.trainer3d.Trainer3D()
# select feature function
ol.feature_function = externfv
# select classifier
ol.cl = imtools.ml.gmmcl.GMMCl(n_components=3)

for one in imtools.datasets.sliver_reader("*[1-9].mhd", read_seg=True):
    numeric_label, vs_mm, oname, orig_data, rname, ref_data = one
    ol.add_train_data(orig_data, ref_data, voxelsize_mm=vs_mm)

ol.fit()



# Testing

In [14]:
one = list(imtools.datasets.sliver_reader("*000.mhd", read_seg=True))[0]
numeric_label, vs_mm, oname, orig_data, rname, ref_data = one
fit = ol.predict(orig_data, voxelsize_mm=vs_mm)

# visualization
plt.figure(figsize=(15,10))
sed3.show_slices(orig_data, fit, slice_step=20, axis=1, flipV=True)

IndexError: list index out of range